In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# ML
from kernels import gaussian_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge
from sklearn.kernel_model import KernelRidge
from sklearn.preprocessing import KernelCenterer
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV
from sklearn.pipeline import Pipeline

# Utilities
from tqdm.notebook import tqdm
import project_utils as utils

In [2]:
# Load train and test sets
train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/train.idxs', dtype=int)
cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_5.idxs', dtype=int)

In [3]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [6]:
# Make directory in which to store models
if not os.path.exists('../Processed_Data/Models'):
    os.mkdir('../Processed_Data/Models')

In [8]:
# Set property names for loading
property_names = ['volumes', 'energies']

# Load structure properties
structure_properties = {}
for pn in property_names:
    structure_properties[pn] = np.loadtxt(f'../Processed_Data/DEEM_330k/Data/structure_{pn}.dat')

# Linear Ridge Regression

In [ ]:
# Set ranges of regularization
regularizations = np.logspace(-3, 2, 6) #np.logspace(-3, 2, 11)
parameter_grid = dict(lr__alpha=regularizations)

In [ ]:
# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=False):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_10k/Data/{cutoff}'
    
    # Set working directory
    work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR'
    if not os.path.exists(work_dir):
        os.mkdir(work_dir)
    
    # Read SOAPs in training set
    soaps = load_hdf5(f'{data_dir}/soaps_power_full_nonorm.hdf5', datasets=None, concatenate=False)
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        
        # Load the property values (just from the train set)
        Y = structure_properties[pn][train_idxs]
         
        # TODO: use TransformedTargetRegressor here to center/scale Y
        lr = Ridge()
        norm_scaler = utils.NormScaler()
        pipeline = Pipeline([
            ('norm_scaler', norm_scaler), 
            ('lr', lr)
        ])
        gscv = GridSearchCV(
            lr, parameter_grid, 
            scoring=[
                'neg_root_mean_squared_error', 
                'neg_mean_absolute_error'
            ],
            cv=utils.cv_generator(cv_idxs),
            refit=False, return_train_score=True
        )
        gscv.fit(soaps)
        # TODO: save CV run in JSON

In [12]:
# Extract optimal parameters

# Loop over cutoffs
for cutoff in cutoffs:
    
    work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR'
    
    # Loop over properties
    for pn in property_names:
        errors_list = np.loadtxt(f'{work_dir}/{pn}_optimization.dat', dtype=dt)
        
        # Loop over error types
        for error in ('mae', 'rmse'):
        
            # TODO: Extract set of parameters corresponding to the minimum error

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error.upper()}-----')
            print('Avg. Error =', np.mean(errors_list[f'{error}_validate'][idx]))
            print('Gamma =', errors_list[idx]['gamma'])
            print('Sigma =', errors_list[idx]['sigma'])
            print('Regularization =', errors_list[idx]['reg'])
            print('')

            # TODO: Save optimal parameters for easy access
            with open(f'{work_dir}/{pn}_{error}_parameters.json', 'w') as f:
                json.dump(opt_params, f)

-----Optimal Parameters for 3.5 volumes MAE-----
Avg. Error = 2.5304993519468213
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 volumes RMSE-----
Avg. Error = 3.654212004317217
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 energies MAE-----
Avg. Error = 0.689774447323908
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 3.5 energies RMSE-----
Avg. Error = 0.9967709902316922
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes MAE-----
Avg. Error = 1.0099737440112349
Gamma = 10.0
Sigma = 1.0
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes RMSE-----
Avg. Error = 1.7406761608394494
Gamma = 1.0
Sigma = 0.01
Regularization = 0.01

-----Optimal Parameters for 6.0 energies MAE-----
Avg. Error = 0.47347945338816533
Gamma = 1.0
Sigma = 0.001
Regularization = 0.001

-----Optimal Parameters for 6.0 energies RMSE-----
Avg. Error = 0.73099440698692

# Kernel Ridge Regression

In [5]:
# Set ranges of kernel gamma (for Gaussian kernel) and regularization
gammas = np.logspace(-3, 2, 6) #np.logspace(-3, 2, 11)
regularizations = np.logspace(-3, 2, 6) #np.logspace(-3, 2, 11)
parameter_grid = dict(krr__alpha=regularizations, kernel_constructor__gamma=gammas)

In [ ]:
# Optimize hyperparameters
# TODO: convert to LR and optimize only regularization

# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=False):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_10k/Data/{cutoff}'
    
    # Set working directory
    work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR'
    if not os.path.exists(work_dir):
        os.mkdir(work_dir)
    
    # Read SOAPs in training set
    soaps = load_hdf5(f'{data_dir}/soaps_power_full_nonorm.hdf5', datasets=None, concatenate=False)
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        
        # Load the property values (just from the train set)
        Y = structure_properties[pn][train_idxs]
        
        krr = KernelRidge(kernel='precomputed')
        kernel_norm_scaler = utils.KernelNormScaler()
        kernel_constructor = utils.KernelConstructor()
        pipeline = Pipeline([
            ('kernel_constructor', kernel_constructor)
            ('kernel_norm_scaler', kernel_norm_scaler), 
            ('krr', krr)
        ])
        gscv = GridSearchCV(
            krr, parameter_grid, 
            scoring=[
                'neg_root_mean_squared_error', 
                'neg_mean_absolute_error'
            ],
            cv=utils.cv_generator(cv_idxs),
            refit=False, return_train_score=True
        )
        gscv.fit(soaps)
        # TODO: save CV run in JSON

In [12]:
# Extract optimal parameters

# Loop over cutoffs
for cutoff in cutoffs:
    
    work_dir = f'../Processed_Data/Models/{cutoff}/Kernel_Models/Gaussian/KRR'
    
    # Loop over properties
    for pn in property_names:
        errors_list = np.loadtxt(f'{work_dir}/{pn}_optimization.dat', dtype=dt)
        
        # Loop over error types
        for error in ('mae', 'rmse'):
        
            # TODO: Extract set of parameters corresponding to the minimum error

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error.upper()}-----')
            print('Avg. Error =', np.mean(errors_list[f'{error}_validate'][idx]))
            print('Gamma =', errors_list[idx]['gamma'])
            print('Sigma =', errors_list[idx]['sigma'])
            print('Regularization =', errors_list[idx]['reg'])
            print('')

            # TODO: Save optimal parameters for easy access
            with open(f'{work_dir}/{pn}_{error}_parameters.json', 'w') as f:
                json.dump(opt_params, f)

-----Optimal Parameters for 3.5 volumes MAE-----
Avg. Error = 2.5304993519468213
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 volumes RMSE-----
Avg. Error = 3.654212004317217
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 energies MAE-----
Avg. Error = 0.689774447323908
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 3.5 energies RMSE-----
Avg. Error = 0.9967709902316922
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes MAE-----
Avg. Error = 1.0099737440112349
Gamma = 10.0
Sigma = 1.0
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes RMSE-----
Avg. Error = 1.7406761608394494
Gamma = 1.0
Sigma = 0.01
Regularization = 0.01

-----Optimal Parameters for 6.0 energies MAE-----
Avg. Error = 0.47347945338816533
Gamma = 1.0
Sigma = 0.001
Regularization = 0.001

-----Optimal Parameters for 6.0 energies RMSE-----
Avg. Error = 0.73099440698692